In [ ]:
#Mounting on the drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
folder_path = '/content/drive/MyDrive/FML_Project'
print(folder_path)

/content/drive/MyDrive/FML_Project


In [ ]:
import os
if os.path.isdir(folder_path):
    print("Folder exists")
else:
    print("Folder doesn't exist")

# List all files in the folder
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    print(file_path)

Folder exists
/content/drive/MyDrive/FML_Project/df_prime_test.csv
/content/drive/MyDrive/FML_Project/df_prime_train.csv
/content/drive/MyDrive/FML_Project/Prime_FULL
/content/drive/MyDrive/FML_Project/resnet50_features.pkl
/content/drive/MyDrive/FML_Project/resnet50_features_testfix.pkl
/content/drive/MyDrive/FML_Project/resnet50_features_final.pkl


In [ ]:
class NotebookArgs:
    def __init__(self, annot_test_prime = "/content/drive/MyDrive/FML_Project/df_prime_test.csv" , annot_train_prime = "/content/drive/MyDrive/FML_Project/df_prime_train.csv", data_root = "/content/drive/MyDrive/FML_Project"):
        self.annot_train_prime = annot_train_prime
        self.annot_test_prime = annot_test_prime
        self.data_root = data_root
args = NotebookArgs()

In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset
from tqdm import tqdm
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import argparse
import os
import copy

import torch
from torch.utils.data import DataLoader
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
import cv2
from skimage.feature import hog

In [ ]:
LABELS_Severity = {35: 0,
                   43: 0,
                   47: 1,
                   53: 1,
                   61: 2,
                   65: 2,
                   71: 2,
                   85: 2}
mean = (.1706)
std = (.2112)
normalize = transforms.Normalize(mean=mean, std=std)

transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    lambda x : x.expand(3,*x.shape[1:]),
    normalize,
])

In [ ]:
class OCTDataset(Dataset):
    def __init__(self, args, subset='train', transform=None,):
        if subset == 'train':
            self.annot = pd.read_csv(args.annot_train_prime)
        elif subset == 'test':
            self.annot = pd.read_csv(args.annot_test_prime)
            
        self.annot['Severity_Label'] = [LABELS_Severity[drss] for drss in copy.deepcopy(self.annot['DRSS'].values)] 
        # print(self.annot)
        self.root = os.path.expanduser(args.data_root)
        self.transform = transform
        # self.subset = subset
        self.nb_classes=len(np.unique(list(LABELS_Severity.values())))
        self.path_list = self.annot['File_Path'].values
        self._labels = self.annot['Severity_Label'].values
        assert len(self.path_list) == len(self._labels)
        # idx_each_class = [[] for i in range(self.nb_classes)]

    def __getitem__(self, index):
        img, target = Image.open(self.root+self.path_list[index]).convert("L"), self._labels[index]

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self._labels)         

In [ ]:
#Loading the dataset
trainset = OCTDataset(args, 'train', transform=transform)
testset = OCTDataset(args, 'test', transform=transform)

# Define hyperparameters
batch_size = 128
num_epochs = 1

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
#implementing the resnet50 feature extractor
device = torch.device('cuda:0')
resnet50 = torchvision.models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(resnet50.children())[:-1]).eval().to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 130MB/s]


In [ ]:
def get_X_y_from_loader(loader):
    all_features = []
    all_labels = []
    for local_batch, local_labels in tqdm(loader):
        local_batch = local_batch.to(device)
        batched_features = model(local_batch).detach().cpu()
        flat_features = [image.flatten().numpy() for image in batched_features]
        flat_labels = [label.item() for label in local_labels]
        all_features.extend(flat_features)
        all_labels.extend(flat_labels)
    return all_features, all_labels

In [ ]:
X_train, y_train = get_X_y_from_loader(train_loader)
X_test, y_test = get_X_y_from_loader(test_loader)

100%|██████████| 63/63 [15:23<00:00, 14.66s/it]


In [ ]:
#training on the Naive bayes Classifier
from sklearn.naive_bayes import GaussianNB
# Train the Naive Bayes classifier
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB()

In [ ]:
y_pred = clf.predict(X_test)

from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, f1_score

# Calculate balanced accuracy score
balanced_acc = balanced_accuracy_score(y_test, y_pred)
print("Balanced accuracy score:", balanced_acc)

# Calculate precision score
precision = precision_score(y_test, y_pred,average='weighted')
print("Precision score:", precision)

# Calculate recall score
recall = recall_score(y_test, y_pred,average='weighted')
print("Recall score:", recall)

# Calculate f1 score
f1 = f1_score(y_test, y_pred,average='weighted')
print("F1 score weighted:", f1)

# Calculate f1 score
f1 = f1_score(y_test, y_pred,average='micro')
print("F1 score Micro:", f1)

# Calculate f1 score
f1 = f1_score(y_test, y_pred,average='macro')
print("F1 score Macro:", f1)

Balanced accuracy score: 0.3530337941628264
Precision score: 0.39645086412144553
Recall score: 0.37423312883435583
F1 score weighted: 0.377640946464049
F1 score Micro: 0.37423312883435583
F1 score Macro: 0.3443498074498262


In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
labels = [0, 1, 2]  # label names

cm = confusion_matrix(y_test, y_pred, labels=labels)

cm_df = pd.DataFrame(cm, index=labels, columns=labels)

print(cm_df)

      0     1    2
0  1261   672  615
1  1632  1422  866
2   435   778  306
